In [12]:
# Hyundai Heavy Industries is one of the world's largest ship manufacturing companies and 
# builds cruise liners. In this project I want to build a predictive model(regression model)
# that will help this company to predict how many crew members will be needed for future ships. 

# Here is what the data looks like so far:

# Description: Measurements of ship size, capacity, crew, and age for 158 cruise
# ships.

# Variables/Columns
# Ship Name     1-20
# Cruise Line   21-40
# Age (as of 2013)   46-48
# Tonnage (1000s of tons)   50-56
# passengers (100s)   58-64
# Length (100s of feet)  66-72
# Cabins  (100s)   74-80
# Passenger Density   82-88
# Crew  (100s)   90-96

# It is saved in a csv file for you called "cruise_ship_info.csv". 

# The client also mentioned that they have found that particular cruise lines will differ in 
# acceptable crew counts, so it is most likely an important feature to include in my analysis.

In [62]:
#Import the findspark module and initialize it with the specified Spark path
import findspark
findspark.init('/home/mina/python-spark/spark-3.4.0-bin-hadoop3/')

#Import the pyspark module and the SparkSession class
import pyspark
from pyspark.sql import SparkSession

#Create a Spark session with the specified app name
spark = SparkSession.builder.appName('Ship_Project').getOrCreate()

In [63]:
#Read a CSV file named 'cruise_ship_info.csv' into a DataFrame
#The 'inferSchema=True' option infers data types for columns, and 'header=True' treats the first row as column names
dataset = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

#Print the schema of the dataset
dataset.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [64]:
# Retrieve and display the first 3 rows of the dataset
dataset.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)]

In [65]:
#Iterate over the first 3 rows of the dataset and print each row to underestand that cruise lines may have an effect on how many crew memebers we need
for record in dataset.head(3):
    print(record , '\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55) 

Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55) 

Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7) 



In [66]:
#Group the DataSet by the 'Cruise_line' column, count cruise line we have in dataset
#Some of Cruise_line is more important than others
dataset.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [67]:
# Import the necessary module for string indexing
from pyspark.ml.feature import StringIndexer

# Create a StringIndexer transformer to index the 'Cruise_line' column
indexer = StringIndexer(inputCol="Cruise_line" , outputCol="Cruise_line_Indexed")

# Fit the indexer model to the dataset to generate an indexing model
indexerModel = indexer.fit(dataset)

# Transform the dataset using the indexing model to add the indexed column
index_df = indexerModel.transform(dataset)

# Display the first 3 rows of the DataFrame after string indexing
index_df.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_Indexed=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_Indexed=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, Cruise_line_Indexed=1.0)]

In [68]:
#Print the schema of the DataFrame after string indexing
index_df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_Indexed: double (nullable = false)



In [69]:
#Retrieve and display the column names of the DataFrame after string indexing
index_df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_Indexed']

In [70]:
#Import the necessary modules for creating feature vectors and vector assembly
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

#Create a VectorAssembler to assemble selected columns into a feature vector
assembler = VectorAssembler(inputCols=['Cruise_line_Indexed',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density'] , outputCol='ShipFeatures')

# Transform the DataFrame using the VectorAssembler to add the 'ShipFeatures' column
output = assembler.transform(index_df)

# Display the first row of the transformed DataFrame
output.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_Indexed=16.0, ShipFeatures=DenseVector([16.0, 6.0, 30.277, 6.94, 5.94, 3.55, 42.64]))]

In [71]:
# Select the 'shipFeatures' and 'crew' columns from the transformed DataFrame
final_data = output.select('ShipFeatures', 'crew')
final_data.show()

+--------------------+----+
|        ShipFeatures|crew|
+--------------------+----+
|[16.0,6.0,30.2769...|3.55|
|[16.0,6.0,30.2769...|3.55|
|[1.0,26.0,47.262,...| 6.7|
|[1.0,11.0,110.0,2...|19.1|
|[1.0,17.0,101.353...|10.0|
|[1.0,22.0,70.367,...| 9.2|
|[1.0,15.0,70.367,...| 9.2|
|[1.0,23.0,70.367,...| 9.2|
|[1.0,19.0,70.367,...| 9.2|
|[1.0,6.0,110.2389...|11.5|
|[1.0,10.0,110.0,2...|11.6|
|[1.0,28.0,46.052,...| 6.6|
|[1.0,18.0,70.367,...| 9.2|
|[1.0,17.0,70.367,...| 9.2|
|[1.0,11.0,86.0,21...| 9.3|
|[1.0,8.0,110.0,29...|11.6|
|[1.0,9.0,88.5,21....|10.3|
|[1.0,15.0,70.367,...| 9.2|
|[1.0,12.0,88.5,21...| 9.3|
|[1.0,20.0,70.367,...| 9.2|
+--------------------+----+
only showing top 20 rows



In [72]:
# Split the final_data DataFrame into training and testing datasets
# using a split ratio of 70% for training data and 30% for testing data
train_data, test_data = final_data.randomSplit([0.7,0.3])

# Display summary statistics of the 'train_data' DataFrame
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               114|
|   mean| 7.720175438596498|
| stddev|3.5519472367264604|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [73]:
# Display summary statistics of the 'test_data' DataFrame
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                44|
|   mean|  7.98590909090909|
| stddev|3.4072312418926574|
|    min|               1.6|
|    max|              13.6|
+-------+------------------+



In [74]:
# Import the necessary module for linear regression
from pyspark.ml.regression import LinearRegression

# Create a LinearRegression model with specified features, label, and prediction columns
lr = LinearRegression(featuresCol='ShipFeatures', labelCol='crew', predictionCol='CrewNumber')

# Fit the LinearRegression model to the training data
lrModel = lr.fit(train_data)

# Evaluate the model on the test data and store the results
test_result = lrModel.evaluate(test_data)

# Display the residuals (differences between predicted and actual values) of the model on the test data
test_result.residuals.show()

23/09/09 23:42:42 WARN Instrumentation: [af5442bb] regParam is zero, which might cause numerical instability and overfitting.


+--------------------+
|           residuals|
+--------------------+
| -1.2832082087000796|
|-0.36674477869317634|
| -0.4413874136254563|
| -1.0747619492568568|
| -0.6190580038826088|
| -0.6364773220564572|
|  0.6786807107614177|
| -0.5857625919210161|
| -1.1443710253007975|
|  0.5566916941008877|
|  0.6197398016191507|
| 0.04062398005324397|
|  1.1025267634927687|
|  1.1025267634927687|
| -0.1298853897840999|
|  0.9850380555621587|
|  0.2774362107835606|
| 0.19201677490648983|
|  0.5590047345969484|
| -0.4167243653939874|
+--------------------+
only showing top 20 rows



In [75]:
# Access and display the root mean squared error (RMSE) from the test_result
test_result.rootMeanSquaredError

0.7636474247531375

In [76]:
# Access and display the R-squared (coefficient of determination) from the test_result
test_result.r2

0.9485995660633993

In [77]:
# Display summary statistics of the 'final_data' DataFrame
final_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [78]:
from pyspark.sql.functions import corr

# Calculate and display the correlation between 'crew' and 'passengers' columns from main dataset
dataset.select(corr('crew' , 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [79]:
# Calculate and display the correlation between 'crew' and 'cabins' columns from main dataset
dataset.select(corr('crew' , 'cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

